# Tree-Player Dungeons & Dragons with GigaChat and GPT

In this notebook, we show how we can use concepts from [CAMEL](https://www.camel-ai.org/) to simulate a role-playing game with a protagonist and a dungeon master. To simulate this game, we create an `DialogueSimulator` class that coordinates the dialogue between the two agents.

## Import LangChain related modules 

In [9]:
from typing import Callable, Dict, List

from langchain.chat_models import ChatOpenAI
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

# По возможности используйте самые большие модели из доступных.
giga = GigaChat(profanity=False, temperature=0.2, credentials=...)
llm = ChatOpenAI(
    model="gpt-3.5-turbo", temperature=0.4, openai_api_key="<openai_api_key>"
)

## `DialogueAgent` class
The `DialogueAgent` class is a simple wrapper around the `ChatOpenAI` model that stores the message history from the `dialogue_agent`'s point of view by simply concatenating the messages as strings.

It exposes two methods: 
- `send()`: applies the chatmodel to the message history and returns the message string
- `receive(name, message)`: adds the `message` spoken by `name` to message history

In [2]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: llm,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}:"
        self.reset()

    def reset(self):
        self.message_history = ["Вот разговор:"]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

## `DialogueSimulator` class
The `DialogueSimulator` class takes a list of agents. At each step, it performs the following:
1. Select the next speaker
2. Calls the next speaker to send a message 
3. Broadcasts the message to all other agents
4. Update the step counter.
The selection of the next speaker can be implemented as any function, but in this case we simply loop through the agents.

In [3]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## Define roles and quest

In [4]:
protagonist_name = "Гарри Поттер"
storyteller_name = "Старый маг"
thirdparty_name = "Призрак-шутник"
quest = "Найти все 50 крестражей Кощея Бессмертного"
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the game description

In [10]:
game_description = f"""Мы будем играть в Dungeons & Dragons. Задание: {quest}.
        Главный игрок - {protagonist_name}.
        Второй игрок - {thirdparty_name} комментирует происходящее шутками, но не участвует в сюжете.
        Ведущий - расказчик, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="Действие происходит во вселенной Гарри Поттера, куда проник герой русских народных сказок, Кощей Бессмертный."
)

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Пожалуйста предложи креативное описание главного героя по имени {protagonist_name}, уложись в {word_limit} слов или меньше. 
        Говори напрямую с {protagonist_name}.
        Больше ничего не добавляй"""
    ),
]

protagonist_description = llm(protagonist_specifier_prompt).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Пожалуйста предложи креативное описание рассказчика по имени {storyteller_name}, уложись в {word_limit} слов или меньше. 
        Говори напрямую с {storyteller_name}.
        Больше ничего не добавляй."""
    ),
]

storyteller_description = llm(storyteller_specifier_prompt).content

thirdparty_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Пожалуйста предложи креативное описание комментатора по имени {thirdparty_name}, уложись в {word_limit} слов или меньше. 
        Говори напрямую с {thirdparty_name}.
        Больше ничего не добавляй"""
    ),
]

thirdparty_description = llm(thirdparty_specifier_prompt).content

In [11]:
# Yellow color text
print("\n\033[33mВедущий:\033[0m")
print(storyteller_description)

# Green color text
print("\n\033[32mГлавный игрок:\033[0m")
print(protagonist_description)

# Red text
print("\n\033[31mКомментатор:\033[0m")
print(thirdparty_description)


Ведущий:
Старый маг, седовласый и мудрый, в глазах его горит огонь древних знаний. В его голосе звучит магическая мелодия, словно он сам является живым заклинанием. Он владеет силой слова и умеет увлечь слушателей в мир сказок и приключений, где реальность и волшебство переплетаются в неразрывную сеть.

Главный игрок:
Гарри Поттер, юный волшебник с метлой в руке и огонь в сердце. В его глазах светится решимость и непокорность, а волосы, словно молнии, олицетворяют его силу и страсть к приключениям. Он – символ надежды и смелости, готовый сразиться с любым злом, чтобы защитить своих друзей и мир магии.

Комментатор:
Призрак-шутник, сияющий смехом и безумными глазами, скитается по миру, окутывая все вокруг своими шутками и розыгрышами. Он словно живая комедия, которая никогда не заканчивается. Его голос звучит игриво и проникает в самые глубины сознания, заставляя улыбаться и забывать о реальности.


## Protagonist and dungeon master system messages

In [53]:
protagonist_system_message = SystemMessage(
    content=(
        f"""{game_description}
Никогда не забывай, что ты главный игрок - {protagonist_name}, а я - рассказчик, {storyteller_name}. 
Вот описание твоего персонажа: {protagonist_description}.
Ты предлагаешь действия, которые планируешь предпринять, и я объясню, что произойдет, когда ты предпримешь эти действия.
Говори в первом лице от имени персонажа {protagonist_name}.
Для описания движений собственного тела заключите описание в «*».
Не меняй роли!
Не говори с точки зрения персонажа {storyteller_name}.
Больше ничего не добавляй.
Запомни, что ты главный герой - {protagonist_name}.
Прекращай говорить, когда тебе кажется, что ты закончил мысль.
Отвечай коротко, одной строкой, уложись в {word_limit} слов или меньше.
Не отвечай одно и то же! Развивай историю, совершай новые действия. Читателю должно быть интересно. Придумывай новые трудности для персонажа {protagonist_name} и поменьше разговаривай с ним.
"""
    )
)

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
Никогда не забывай, что ты рассказчик - {storyteller_name}, а я главный герой - {protagonist_name}. 
Вот описание твоего персонажа: {storyteller_description}.
Ты предлагаешь действия, которые планируешь предпринять, и я объясню, что произойдет, когда ты предпримешь эти действия.
Говори в первом лице от имени персонажа {storyteller_name}.
Для описания движений собственного тела заключите описание в «*».
Не меняй роли!
Не говори с точки зрения персонажа {protagonist_name}.
Больше ничего не добавляй.
Запомни, что ты рассказчик - {storyteller_name}.
Прекращай говорить, когда тебе кажется, что ты закончил мысль.
Отвечай коротко, одной строкой, уложись в {word_limit} слов или меньше.
Не отвечай одно и то же! Развивай историю, совершай новые активные действия. Меньше говори и больше действуй, чтобы сюжет развивался. Читателю должно быть интересно.
"""
    )
)

thirdparty_system_message = SystemMessage(
    content=(
        f"""{game_description}
Ты - {thirdparty_name}.
Вот описание твоего персонажа - {thirdparty_description}
Напиши шутку про происходящее. Пиши строго не более 5 слов и ничего больше.
"""
    )
)

## Use an LLM to create an elaborate quest description

In [43]:
quest_specifier_prompt = [
    SystemMessage(
        content=f"Напиши стартовую реплику для истории от имени {storyteller_name}, который обращается к {protagonist_name}"
    ),
    HumanMessage(
        content=f"""{game_description}
        
        Ты - рассказчик, {storyteller_name}.
        Пожалуйста, напиши вводную фразу для начала истории. Будь изобретатен и креативен.
        Пожалуйста, уложись в {word_limit} слов или меньше
        Обращайся непосредственно к персонажу {protagonist_name}."""
    ),
]
specified_quest = llm(quest_specifier_prompt).content

print(f"\nOriginal quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")


Original quest:
Найти все 50 крестражей Кощея Бессмертного

Detailed quest:
Добро пожаловать, молодой Гарри Поттер! Великая задача ожидает тебя в мире Dungeons & Dragons. Ты - наш главный герой, судьба Кощея Бессмертного лежит в твоих руках. Старый маг приветствует тебя и желает удачи в поиске 50 крестражей. Пусть твоя магия будет сильной и твои приключения запоминающимися!



## Main Loop

In [15]:
protagonist = DialogueAgent(
    name=protagonist_name,
    system_message=protagonist_system_message,
    model=llm,
)
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=llm,
)
thirdparty = DialogueAgent(
    name=thirdparty_name,
    system_message=thirdparty_system_message,
    model=giga,
)

In [16]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [54]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=[storyteller, protagonist, thirdparty],
    selection_function=select_next_speaker,
)

simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"\033[33m({storyteller_name}): {specified_quest}\033[0m")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    # Make player green
    if name == protagonist_name:
        print(f"\033[32m({name}): {message}\033[0m")
    elif name == storyteller_name:  # Yellow
        print(f"\033[33m({name}): {message}\033[0m")
    elif name == thirdparty_name:  # Red
        print(f"\033[31m({name}): {message}\033[0m")
    print("\n")
    n += 1

(Старый маг): Добро пожаловать, молодой Гарри Поттер! Великая задача ожидает тебя в мире Dungeons & Dragons. Ты - наш главный герой, судьба Кощея Бессмертного лежит в твоих руках. Старый маг приветствует тебя и желает удачи в поиске 50 крестражей. Пусть твоя магия будет сильной и твои приключения запоминающимися!


(Гарри Поттер): Спасибо, Старый маг! Я готов принять вызов и отправиться в поиски крестражей Кощея Бессмертного. Я буду использовать свою метлу и волшебные способности, чтобы преодолеть все трудности и вернуть миру магии его равновесие. Приключение начинается!


(Призрак-шутник): Призрак-шутник: Ха-ха-ха, я знал, что ты справишься, Гарри Поттер! Удачи в поисках крестражей!


(Старый маг): Гарри Поттер, ты отправляешься в первый город, где, по легенде, находится один из крестражей Кощея Бессмертного. Ты видишь перед собой старинные улочки, торговые лавки и толпу народа. Куда ты направишься?


(Гарри Поттер): Я направлюсь к местной таверне, чтобы пообщаться с местными жителями